# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [46]:
import pandas as pd
from nltk import word_tokenize
descr = pd.read_csv('preprocessed_descriptions.csv')
descr.fillna('', inplace=True)
words = []

for i in range(len(descr)):
    words += (word_tokenize(descr['preprocessed_descriptions'][i], language='English'))
words = set(words)

print(words)

{'things', 'taylortwo', 'rztraminer', '244592', 'sugars', 'messageboard', 'blended', 'richest', 'pc', 'traced', 'prettiest', 'throwdown', 'dolly', 'delishes', 'camping', 'introducing', 'lemonade', 'cheaply', 'ironically', 'applebees', 'babara', 'kumera', 'provides', 'befits', 'creme', 'thermos', 'nm', 'steeped', 'tom921', 'kitchen', 'mingle', 'bundled', 'pried', 'coordinating', 'throughly', 'schweinerippen', 'lantana', 'segan', 'santorini', 'porter', 'notebook', 'repeat', 'imported', 'dp', 'use', 'granitas', 'aunt', 'americans', 'bagel', 'indigo', 'nantucket', 'dynamite', 'berne', 'cognac', 'lobsta', 'levana', 'philadelpha', 'bao', 'chuckwagon', 'redolent', 'concocting', 'dairyland', 'yummby', 'alleghany', 'whala', 'quickie', 'lena', 'doorstep', 'ghouls', 'blueberries', 'nigiri', 'lines', 'fieri', 'equates', 'christmasy', '21', 'painting', 'doeas', 'shawarma', 'walking', 'crannberries', 'orchards', 'esl', 'satiety', 'healthified', 'recrisp', 'rioja', 'nuoc', 'dagoba', 'hickory', 'sucan

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [87]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
snb_stemmer_ru = SnowballStemmer('english')

stemmed_words = []
normalized_words = []

for i in words:
    stemmed_words.append(snb_stemmer_ru.stem(i))
    normalized_words.append(lemmatizer.lemmatize(i))

res = pd.DataFrame({'words': list(words),
                    'stemmed_word': stemmed_words,
                   'normalized_words': normalized_words})
res = res.set_index('words')
res

,stemmed_word,normalized_words
words,,
things,thing,thing
taylortwo,taylortwo,taylortwo
rztraminer,rztramin,rztraminer
244592,244592,244592
sugars,sugar,sugar
...,...,...
worldwiderecipes,worldwiderecip,worldwiderecipes
proofing,proof,proofing
appoximate,appoxim,appoximate


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [113]:
from nltk.corpus import stopwords
from nltk.probability import FreqDist
stop_words = stopwords.words('english')

tokens = []

for i in range(len(descr)):
    tokens += (word_tokenize(descr['preprocessed_descriptions'][i], language='English'))

fdist = FreqDist(tokens)

sw_cnt = 0
for i in tokens:
    if i in stop_words:
        sw_cnt +=1

print(f'Топ-10 самых часто употребляемых слов {fdist.most_common(10)}', end ='\n\n')     
print(f'Обработано токенов:{fdist.N()}; Доля стоп-слов составляет:{round((sw_cnt/fdist.N())*100, 2)}%', end ='\n\n')

ntokens = [i for i in tokens if i not in stop_words]
cdist = FreqDist(ntokens)
print(f'Топ-10 самых часто употребляемых слов (без учета стоп-слов) {cdist.most_common(10)}') 

Топ-10 самых часто употребляемых слов [('the', 40413), ('a', 35131), ('and', 30585), ('i', 27945), ('this', 27181), ('to', 23598), ('it', 23300), ('is', 20306), ('of', 18405), ('for', 16023)]

Обработано токенов:1103895; Доля стоп-слов составляет:47.12%

Топ-10 самых часто употребляемых слов (без учета стоп-слов) [('recipe', 15198), ('make', 6438), ('time', 5287), ('use', 4652), ('great', 4522), ('like', 4276), ('easy', 4263), ('one', 4018), ('good', 3887), ('made', 3874)]


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).